In [ ]:
# ==============================================================================
# HUBUNGKAN GOOGLE DRIVE DI AWAL SESI
# ==============================================================================
from google.colab import drive

try:
    drive.mount('/content/drive')
    print("✅ Google Drive berhasil terhubung di awal sesi.")
except Exception as e:
    print(f"   ❌ Gagal menghubungkan Google Drive. Error: {e}")
    raise SystemExit("Proses dihentikan.")

In [ ]:
!nvidia-smi

In [ ]:
# ==============================================================================
# 1. INSTALASI DAN IMPORT LIBRARY
# ==============================================================================
print("🚀 Menginstal library yang dibutuhkan...")
!pip install ultralytics pandas -q

import os
import random
import yaml
from getpass import getpass
from pathlib import Path
from tqdm.notebook import tqdm
from google.colab import userdata

print("\n✅ Library berhasil diinstal dan diimpor.")

In [ ]:
# ==============================================================================
# TRANSFER DATASET DARI NEXTCLOUD MENGGUNAKAN RCLONE (VERSI PERBAIKAN)
# ==============================================================================
import time
import subprocess # Import library yang dibutuhkan

print("🚀 Memulai proses transfer dataset dengan rclone...")
start_total_time = time.time()

# 1. Instal rclone (alat transfer file cloud)
print("\n[1/4] Menginstal rclone...")
!curl https://rclone.org/install.sh | sudo bash > /dev/null 2>&1
print("   ✅ rclone berhasil diinstal.")

# 2. Konfigurasi rclone secara otomatis menggunakan Colab Secrets
print("\n[2/4] Mengkonfigurasi rclone untuk Nextcloud...")
try:
    NEXTCLOUD_HOSTNAME = userdata.get("NEXTCLOUD_HOSTNAME")
    NEXTCLOUD_USERNAME = userdata.get("NEXTCLOUD_USERNAME")
    NEXTCLOUD_PASSWORD = userdata.get("NEXTCLOUD_PASSWORD")

    if not NEXTCLOUD_HOSTNAME.startswith(('http://', 'https://')):
        NEXTCLOUD_HOSTNAME = 'https://' + NEXTCLOUD_HOSTNAME

    # 1. Jalankan 'rclone obscure' menggunakan subprocess untuk mendapatkan password terenkripsi
    obscured_password_bytes = subprocess.check_output(['rclone', 'obscure', NEXTCLOUD_PASSWORD])
    obscured_password = obscured_password_bytes.decode('utf-8').strip()
    # ------------------------------------

    # 2. Gunakan password yang sudah terenkripsi di dalam file konfigurasi
    rclone_config_content = f"""
[nextcloud]
type = webdav
url = https://drive.cloud.alfa.cc/remote.php/dav/files/{NEXTCLOUD_USERNAME}/
vendor = nextcloud
user = {NEXTCLOUD_USERNAME}
pass = {obscured_password}
"""
    rclone_config_path = Path("/root/.config/rclone/rclone.conf")
    rclone_config_path.parent.mkdir(parents=True, exist_ok=True)
    with open(rclone_config_path, "w") as f:
        f.write(rclone_config_content)
    print("   ✅ Konfigurasi rclone berhasil dibuat.")

except Exception as e:
    print(f"   ❌ Gagal membuat konfigurasi. Pastikan secrets Anda benar. Error: {e}")
    raise SystemExit("Proses dihentikan.")

In [ ]:
## FOR TESTING

remote_base_rclone = "tbe/nanda/fotocompressed/test_rclone"
# -----------------------------

remote_paths = {
    "images_train": f"{remote_base_rclone}/images/train",
    "labels_train": f"{remote_base_rclone}/labels/train",
    "images_val": f"{remote_base_rclone}/images/val",
    "labels_val": f"{remote_base_rclone}/labels/val",
    "classes_file": f"{remote_base_rclone}/keseragaman_rumpun_padi.txt"
}

local_base = Path("/content/dataset_padi_test") # Menggunakan folder tujuan berbeda untuk tes
local_paths = { "images_train": local_base / "images/train", "labels_train": local_base / "labels/train", "images_val": local_base / "images/val", "labels_val": local_base / "labels/val", "classes_file": "/content/classes.txt" }

print(f"\n[3/4] Path telah ditentukan:")
print(f"   - Sumber di Nextcloud : nextcloud:{remote_base_rclone}")
print(f"   - Tujuan di Colab     : {local_base}")

# 4. Jalankan proses sinkronisasi
print("\n[4/4] Memulai sinkronisasi...")
!rclone sync nextcloud:{remote_paths["images_train"]} {local_paths["images_train"]} --progress --transfers=16 --stats-one-line --dry-run -v
!rclone sync nextcloud:{remote_paths["labels_train"]} {local_paths["labels_train"]} --progress --transfers=16 --stats-one-line --dry-run -v
!rclone sync nextcloud:{remote_paths["images_val"]} {local_paths["images_val"]} --progress --transfers=16 --stats-one-line --dry-run -v
!rclone sync nextcloud:{remote_paths["labels_val"]} {local_paths["labels_val"]} --progress --transfers=16 --stats-one-line --dry-run -v
print("   Menyalin file class...")
!rclone copyto nextcloud:{remote_paths['classes_file']} {local_paths['classes_file']} --stats-one-line --dry-run -v

end_total_time = time.time()
print(f"\n🎉 UJI COBA SINKRONISASI SELESAI!")
print(f"   Total waktu yang dibutuhkan: {(end_total_time - start_total_time) / 3600:.2f} menit.")
print(f"   Dataset uji coba Anda sekarang siap di: {local_base}")

In [ ]:
!rm -rf /content/dataset_padi_test
!rm -rf /content/classes.txt

In [ ]:
# 3. Tentukan path sumber di Nextcloud dan tujuan di Colab
remote_base_rclone = "tbe/nanda/fotocompressed/balanced_dataset"
remote_paths = {
    "images_train": f"{remote_base_rclone}/images/train",
    "labels_train": f"{remote_base_rclone}/labels/train",
    "images_val": f"{remote_base_rclone}/images/val",
    "labels_val": f"{remote_base_rclone}/labels/val",
    "classes_file": f"{remote_base_rclone}/keseragaman_rumpun_padi.txt"
}

local_base = Path("/content/dataset_padi")
local_paths = {
    "images_train": local_base / "images/train",
    "labels_train": local_base / "labels/train",
    "images_val": local_base / "images/val",
    "labels_val": local_base / "labels/val",
    "classes_file": "/content/classes.txt"
}

print(f"\n[3/4] Path telah ditentukan:")
print(f"   - Sumber di Nextcloud : nextcloud:{remote_base_rclone}")
print(f"   - Tujuan di Colab     : {local_base}")

In [ ]:
# 4. Jalankan proses sinkronisasi
print("\n[4/4] Memulai sinkronisasi...")
!rclone sync nextcloud:{remote_paths["images_train"]} {local_paths["images_train"]} --progress --transfers=16 --stats-one-line -v
!rclone sync nextcloud:{remote_paths["labels_train"]} {local_paths["labels_train"]} --progress --transfers=16 --stats-one-line -v
!rclone sync nextcloud:{remote_paths["images_val"]} {local_paths["images_val"]} --progress --transfers=16 --stats-one-line -v
!rclone sync nextcloud:{remote_paths["labels_val"]} {local_paths["labels_val"]} --progress --transfers=16 --stats-one-line -v

print("   Menyalin file class...")
!rclone copyto nextcloud:{remote_paths['classes_file']} {local_paths['classes_file']} --stats-one-line -v

end_total_time = time.time()
print(f"\n🎉 SINKRONISASI SELESAI!")
print(f"   Total waktu yang dibutuhkan: {(end_total_time - start_total_time) / 3600:.2f} menit.")
print(f"   Dataset Anda sekarang siap di: {local_base}")

In [ ]:
# ==============================================================================
# FUNGSI UNTUK MENGANALISIS DISTRIBUSI KELAS
# ==============================================================================
from collections import defaultdict
import pandas as pd

def analyze_class_distribution(label_dir_path):
    """Membaca semua file label dalam direktori dan menghitung instance per kelas."""
    class_counts = defaultdict(int)
    if not label_dir_path.exists():
        print(f"⚠️ Peringatan: Direktori tidak ditemukan: {label_dir_path}")
        return class_counts

    for label_file in label_dir_path.glob('*.txt'):
        with open(label_file, 'r') as f:
            for line in f:
                try:
                    # Ambil ID kelas (angka pertama di setiap baris)
                    class_id = int(line.split()[0])
                    class_counts[class_id] += 1
                except (ValueError, IndexError):
                    # Mengabaikan baris yang kosong atau formatnya salah
                    continue
    return class_counts

# ==============================================================================
# MENJALANKAN ANALISIS DAN MENAMPILKAN HASIL
# ==============================================================================
print("\n📊 Menganalisis distribusi kelas pada dataset lokal...")

try:
    # Pertama, baca nama kelas dari file yang sudah diunduh
    with open(local_paths["classes_file"], 'r') as f:
        class_names = [line.strip() for line in f if line.strip()]
    class_name_map = {i: name for i, name in enumerate(class_names)}

    # Analisis folder label train dan val
    train_counts = analyze_class_distribution(local_paths["labels_train"])
    val_counts = analyze_class_distribution(local_paths["labels_val"])

    # Siapkan data untuk ditampilkan dalam tabel yang rapi menggunakan pandas
    distribution_data = {
        'Class ID': [],
        'Class Name': [],
        'Training Set Count': [],
        'Validation Set Count': []
    }

    all_class_ids = sorted(list(set(train_counts.keys()) | set(val_counts.keys())))

    for cid in all_class_ids:
        distribution_data['Class ID'].append(cid)
        distribution_data['Class Name'].append(class_name_map.get(cid, 'Unknown'))
        distribution_data['Training Set Count'].append(train_counts.get(cid, 0))
        distribution_data['Validation Set Count'].append(val_counts.get(cid, 0))

    # Tampilkan tabel
    df = pd.DataFrame(distribution_data)
    print("\n✅ Analisis Selesai. Berikut adalah distribusinya:")
    print("--------------------------------------------------")
    print(df.to_string(index=False))
    print("--------------------------------------------------")

except Exception as e:
    print(f"❌ Gagal menganalisis distribusi kelas. Error: {e}")

In [ ]:
# ==============================================================================
# MEMBACA NAMA KELAS DARI FILE
# ==============================================================================
print("📝 Membuat file konfigurasi dataset (data.yaml)...")

try:
    with open(local_paths["classes_file"], 'r') as f:
        class_names = [line.strip() for line in f if line.strip()]

    num_classes = len(class_names)
    print(f"   Jumlah kelas terdeteksi: {num_classes}")
    print(f"   Nama kelas: {class_names}")

    # ==============================================================================
    # MEMBUAT KONTEN FILE YAML
    # ==============================================================================
    dataset_config = {
        'train': str(local_paths["images_train"].resolve()),
        'val': str(local_paths["images_val"].resolve()),
        'nc': num_classes,
        'names': class_names
    }

    config_path = local_base / "data.yaml"
    with open(config_path, 'w') as f:
        yaml.dump(dataset_config, f, default_flow_style=False, sort_keys=False)

    print(f"\n✅ File konfigurasi berhasil dibuat di: {config_path}")

    # Tampilkan isi file untuk verifikasi
    print("\nIsi file data.yaml:")
    print("---------------------")
    with open(config_path, 'r') as f:
        print(f.read())

except Exception as e:
    print(f"❌ Gagal membuat file konfigurasi. Error: {e}")

In [ ]:
# ==============================================================================
# MENJALANKAN TRAINING SINGKAT
# ==============================================================================
print("\n🏋️‍♂️ Memulai baseline training (sanity check)...")
print("Model: yolo11s.pt")
print("Epochs: 50")
print("Image Size: 640x640")
print("--------------------------------------------------")

!yolo task=detect mode=train model=yolo11s.pt data={local_base}/data.yaml epochs=50 imgsz=640 fraction=0.5 project=runs/padi_detection name=full_training

print("\n✅ Sanity check selesai!")
print("Jika tidak ada error, berarti konfigurasi Anda sudah benar.")
print("Hasil training dapat dilihat di folder 'runs/padi_detection/full_training'")

In [ ]:
!yolo detect val model=/content/runs/padi_detection/full_training/weights/best.pt data={local_base}/data.yaml max_det=2000 classes=1,2

In [ ]:
# ==============================================================================
# MENYIMPAN SELURUH FOLDER 'runs' KE GOOGLE DRIVE
# ==============================================================================
import shutil

print("💾 Mempersiapkan untuk menyimpan seluruh folder 'runs' ke Google Drive...")

# Drive sudah terhubung, jadi kita bisa langsung ke proses penyalinan

# Tentukan folder sumber dan tujuan
source_folder = Path('runs')
drive_save_path = Path('/content/drive/MyDrive/YOLOv8_Padi_Runs')
destination_path = drive_save_path / source_folder.name

# Buat folder tujuan utama di Drive jika belum ada
drive_save_path.mkdir(parents=True, exist_ok=True)

print(f"   📂 Folder sumber: {source_folder}")
print(f"   🎯 Folder tujuan di Drive: {destination_path}")

# Salin folder ke Google Drive
try:
    if not source_folder.exists():
        raise FileNotFoundError("Folder 'runs' tidak ditemukan.")

    if destination_path.exists():
        print(f"   ⚠️ Folder tujuan '{destination_path.name}' sudah ada. Menghapus versi lama...")
        shutil.rmtree(destination_path)

    shutil.copytree(source_folder, destination_path)
    print(f"\n🎉 SUKSES! Seluruh folder '{source_folder.name}' telah disimpan.")

except Exception as e:
    print(f"   ❌ GAGAL: Terjadi error saat menyalin. Error: {e}")